In [7]:
import pandas as pd
import numpy as np


In [8]:
!git clone https://github.com/laxmimerit/IMDB-Movie-Reviews-Large-Dataset-50k.git

fatal: destination path 'IMDB-Movie-Reviews-Large-Dataset-50k' already exists and is not an empty directory.


In [9]:
df = pd.read_excel('/content/IMDB-Movie-Reviews-Large-Dataset-50k/train.xlsx')
df.head()

,Reviews,Sentiment
0,"When I first tuned in on this morning news, I ...",neg
1,"Mere thoughts of ""Going Overboard"" (aka ""Babes...",neg
2,Why does this movie fall WELL below standards?...,neg
3,Wow and I thought that any Steven Segal movie ...,neg
4,"The story is seen before, but that does'n matt...",neg


In [10]:
df.shape

(25000, 2)

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [12]:
from sklearn.model_selection import train_test_split

In [16]:
from sklearn.svm import LinearSVC

In [18]:
!pip install git+https://github.com/laxmimerit/preprocess_kgptalkie.git



  Cloning https://github.com/laxmimerit/preprocess_kgptalkie.git to /tmp/pip-req-build-ptramehi
  Running command git clone --filter=blob:none --quiet https://github.com/laxmimerit/preprocess_kgptalkie.git /tmp/pip-req-build-ptramehi
  Resolved https://github.com/laxmimerit/preprocess_kgptalkie.git to commit 9ca68d37027af9f6a30d54640347ce3b2e2694b3
  Preparing metadata (setup.py) ... done
  Created wheel for preprocess-kgptalkie: filename=preprocess_kgptalkie-0.1.3-py3-none-any.whl size=7620 sha256=c4f73134df46d681e8fae2f6775023579d3cde76f38065695402636ccd220ff6
  Stored in directory: /tmp/pip-ephem-wheel-cache-2gw7qv7y/wheels/5c/94/34/99d5ff65e88b8d9a6c5e8d8652f2311d87790a61a1b7466e21
Successfully built preprocess-kgptalkie


In [28]:
import re
import preprocess_kgptalkie as ps
def get_clean(x):
  x = str(x).lower().replace('\\','').replace('_',' ')
  x = ps.cont_exp(x) #you're -> you are; i'm -> i am
  x = ps.remove_emails(x)
  x = ps.remove_urls(x)
  x = ps.remove_html_tags(x)
  x = ps.remove_accented_chars(x)
  x = ps.remove_special_chars(x)
  x = re.sub('(.)\\1{2,}','\\1',x)
  return x

In [26]:
a = 'aaarrrunnnkummmaaarrr'
a = re.sub('(.)\\1{2,}','\\1',a)
print(a)

arunkumar


In [29]:
df['Reviews'] = df['Reviews'].apply(lambda x:get_clean(x))

In [30]:
# tfidf
tfidf = TfidfVectorizer(max_features = 5000)
X = df['Reviews']
y = df['Sentiment']

X = tfidf.fit_transform(X)


In [31]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=0)

In [32]:
print(X_train.shape)
print(y_train.shape)


(20000, 5000)
(20000,)


In [33]:
print(X_test.shape)
print(y_test.shape)

(5000, 5000)
(5000,)


In [35]:
clf = LinearSVC()
clf.fit(X_train,y_train)

LinearSVC()

In [36]:
y_pred =clf.predict(X_test)


In [37]:
from sklearn.metrics import classification_report

In [38]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

         neg       0.87      0.87      0.87      2480
         pos       0.87      0.88      0.88      2520

    accuracy                           0.87      5000
   macro avg       0.87      0.87      0.87      5000
weighted avg       0.87      0.87      0.87      5000



In [39]:
from sklearn.metrics import accuracy_score

In [40]:
test_score =accuracy_score(y_test,y_pred)

In [45]:
print(test_score)

0.8746


In [46]:
x = 'this good book is really good,thanks a lot of making it.'


In [47]:
x=get_clean(x)
vec = tfidf.transform([x])


In [48]:
vec.shape

(1, 5000)

In [49]:
clf.predict(vec)


array(['pos'], dtype=object)

In [50]:
import pickle

In [52]:
pickle.dump(clf,open('model','wb'))